In [ ]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
# 1. Путь к датасету CLEVR
clevr_dir = 'gdrive/path_to_clevr_dataset'  # Указываем путь к папке с данными CLEVR

# Загрузка данных (вопросы и ответы)
with open(os.path.join(clevr_dir, 'questions/CLEVR_train_questions.json')) as f:
    train_questions = json.load(f)  # Загружаем JSON с вопросами и ответами для тренировочного набора

In [ ]:
# Предобработка вопросов и ответов
questions = [q['question'] for q in train_questions['questions']]  # Извлекаем текст вопросов
answers = [q['answer'] for q in train_questions['questions']]      # Извлекаем текст ответов

# Преобразование текстовых вопросов в числовой формат
tokenizer = keras.preprocessing.text.Tokenizer(num_words=5000)  # Создаем токенайзер с ограничением в 5000 слов
tokenizer.fit_on_texts(questions)                               # Обучаем токенайзер на тексте вопросов

questions_seq = tokenizer.texts_to_sequences(questions)         # Преобразуем вопросы в последовательности чисел (индексы слов)
questions_padded = keras.preprocessing.sequence.pad_sequences(questions_seq, maxlen=30)  # Приводим последовательности к одинаковой длине (30)



In [ ]:
# Загрузка изображений сцены
image_dataset = image_dataset_from_directory(
    os.path.join(clevr_dir, 'images/train'),   # Путь к папке с изображениями тренировочного набора
    image_size=(128, 128),                     # Приводим все изображения к размеру 128x128
    batch_size=32                              # Устанавливаем размер батча
)

In [ ]:
# 2. Определение модели
# CNN для извлечения признаков из изображения
def build_cnn_model():
    cnn_base = tf.keras.applications.ResNet50(           # Используем предобученную модель ResNet50 в качестве основы для CNN
        include_top=False,                               # Исключаем последние слои для классификации
        input_shape=(128, 128, 3),                       # Устанавливаем размер входного изображения
        pooling='avg'                                    # Применяем Global Average Pooling для получения признаков
    )
    cnn_base.trainable = False  # Замораживаем веса модели, чтобы они не обучались на этапе тренировки
    return cnn_base             # Возвращаем базовую модель CNN

In [ ]:
# Трансформер для обработки вопросов
class TransformerBlock(layers.Layer):  # Создаем пользовательский слой для трансформерного блока
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)  # Мультиголовное внимание
        self.ffn = keras.Sequential([                                                  # FFN (Feed Forward Network) для нелинейных преобразований
            layers.Dense(ff_dim, activation="relu"),                                   # Первый слой с активацией ReLU
            layers.Dense(embed_dim)                                                    # Второй слой для восстановления исходной размерности
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)  # Нормализация слоя для стабилизации
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)  # Еще одна нормализация слоя
        self.dropout1 = layers.Dropout(rate)                       # Dropout для регуляризации
        self.dropout2 = layers.Dropout(rate)                       # Dropout после FFN

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)  # Операция внимания: каждая часть текста взаимодействует с каждой другой
        attn_output = self.dropout1(attn_output, training=training)  # Применяем Dropout в режиме обучения
        out1 = self.layernorm1(inputs + attn_output)  # Добавляем резидуальное соединение и нормализуем результат
        ffn_output = self.ffn(out1)  # Пропускаем через FFN (Feed Forward Network)
        ffn_output = self.dropout2(ffn_output, training=training)  # Применяем Dropout во второй раз
        return self.layernorm2(out1 + ffn_output)  # Добавляем резидуальное соединение и нормализуем снова

# Вспомогательный класс для добавления позиционной информации (необходим для трансформера)
class PositionalEncoding(layers.Layer):  # Создаем слой для добавления позиционных кодировок
    def __init__(self, max_len, embed_dim):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(max_len, embed_dim)  # Генерируем позиционные кодировки

    def get_angles(self, pos, i, embed_dim):
        angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(embed_dim))  # Определяем шкалу для позиционного кодирования
        return pos * angle_rates

    def positional_encoding(self, position, embed_dim):
        angle_rads = self.get_angles(np.arange(position)[:, np.newaxis], np.arange(embed_dim)[np.newaxis, :], embed_dim)
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])  # Применяем синус к четным индексам
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])  # Применяем косинус к нечетным индексам
        pos_encoding = angle_rads[np.newaxis, ...]         # Добавляем размерность для батча
        return tf.cast(pos_encoding, dtype=tf.float32)     # Преобразуем к типу float32

    def call(self, x):
        return x + self.pos_encoding[:, :tf.shape(x)[1], :]  # Добавляем позиционное кодирование к входным данным

In [ ]:
# Полная модель
def build_vqa_model(vocab_size, embed_dim, num_heads, ff_dim, num_blocks, num_classes):
    # Вход для изображения
    image_input = layers.Input(shape=(128, 128, 3))     # Входной слой для изображений (размер 128x128x3)
    cnn_model = build_cnn_model()                       # Используем нашу модель CNN для извлечения признаков из изображений
    image_features = cnn_model(image_input)             # Извлекаем признаки изображения

    # Вход для вопроса
    question_input = layers.Input(shape=(30,))          # Входной слой для вопросов (длина последовательности 30)
    embedding = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)(question_input)  # Преобразуем слова в вектора (Embedding)

    # Добавляем позиционное кодирование к векторным представлениям вопросов
    x = PositionalEncoding(30, embed_dim)(embedding)

    # Пропускаем через несколько трансформерных блоков
    for _ in range(num_blocks):
        x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)

    # Объединение признаков изображения и вопроса
    combined = layers.concatenate([image_features, layers.Flatten()(x)])  # Объединяем признаки изображения и вопросы в один вектор

    # Финальный слой классификации для предсказания ответа
    output = layers.Dense(num_classes, activation='softmax')(combined)  # Применяем softmax для классификации ответов

    # Создание модели
    model = keras.Model(inputs=[image_input, question_input], outputs=output)  # Определяем модель с двумя входами (изображение и вопрос)
    return model

In [ ]:
# 3. Компиляция и обучение модели
# Получение количества уникальных ответов для задания количества классов
answer_tokenizer = keras.preprocessing.text.Tokenizer()  # Создаем токенайзер для ответов
answer_tokenizer.fit_on_texts(answers)                   # Обучаем токенайзер на ответах
answers_seq = answer_tokenizer.texts_to_sequences(answers)  # Преобразуем ответы в числовые последовательности
answers_categorical = keras.utils.to_categorical(answers_seq, num_classes=len(answer_tokenizer.word_index) + 1)  # Преобразуем в one-hot encoding

# Построение модели VQA
model = build_vqa_model(vocab_size=5000, embed_dim=64, num_heads=8, ff_dim=128, num_blocks=4, num_classes=len(answer_tokenizer.word_index) + 1)

# Компиляция модели
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # Компилируем модель с Adam и функцией потерь для классификации

# Обучение модели
model.fit(
    [image_dataset, questions_padded],   # Передаем изображения и закодированные вопросы в модель
    answers_categorical,                 # Ответы в формате one-hot
    epochs=10,                           # Количество эпох обучения
    batch_size=32                        # Размер батча
)
